# _Get_ Classind _movies_ _details_


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
RAW_CLASSIND_MOVIES: pd.DataFrame = pd.read_csv(
    "../data/raw/classind-movies.csv",
    header=None,
    names=[
        "id_classind",
        "title",
        "director",
        "category",
        "market",
        "rating_intended",
        "rating_assigned",
    ],
    index_col="id_classind",
    usecols=[1, 2, 7, 11, 13, 15, 16],
    dtype_backend="pyarrow",
)

IDS_ANCINE_CLASSIND: pd.Series = pd.read_csv(
    "../data/ids-ancine-classind.csv",
    index_col="id_ancine",
    dtype_backend="pyarrow",
).squeeze("columns")

In [ ]:
classind_details: pd.DataFrame = pd.DataFrame(None, index=IDS_ANCINE_CLASSIND)

classind_details: pd.DataFrame = classind_details.join(RAW_CLASSIND_MOVIES)

classind_details["rating"] = np.where(
    classind_details["rating_assigned"].isna(),
    classind_details["rating_intended"],
    classind_details["rating_assigned"],
)

classind_details: pd.DataFrame = classind_details.groupby("id_classind").last()

classind_details: pd.DataFrame = classind_details[["rating"]].dropna()

In [ ]:
RATINGS: dict[str, str] = {
    "Exibição em qualquer horário": "livre",
    "Inadequado para menores de 14 anos": "14",
    "Inadequado para menores de 16 anos": "16",
    "Inadequado para menores de 18 anos": "18",
    "livre": "livre",
    "Não recomendada para menores de 12 anos: inadequada para exibição antes das 20 horas": "12",
    "não recomendado para menores de 10 (dez) anos": "10",
    "não recomendado para menores de 12 (doze) anos": "12",
    "não recomendado para menores de 14 (catorze) anos": "14",
    "não recomendado para menores de 16 (dezesseis) anos": "16",
    "não recomendado para menores de 18 (dezoito) anos": "18",
}

classind_details["rating"] = classind_details["rating"].apply(lambda rating: RATINGS.get(rating))

In [ ]:
classind_details.to_csv("../data/movies-details-classind.csv")